# Please predict and score using you’re the best model to testbase.csv data as final submission

### In this notebook , I'm going to predictions customers are eligible for the credit and check whether what are the missing criteria to know why user ineligible in here

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/creditscroring/trainbase.csv')
df_test = pd.read_csv('../input/creditscroring/testbase.csv')


In [ ]:
df_test.info()

In [ ]:
df_train.info()

In [ ]:
#the id column is not useful for us and building a model then we must drop it

df_train.drop(columns='id', inplace = True)
df_test.drop(columns='id', inplace = True)

In [ ]:
df_train.describe().transpose()

In [ ]:
df_train['branchcode'].unique()
df_test['branchcode'].unique()

In [ ]:
df_test['branchcode'].unique()

In [ ]:
import missingno as msno
msno.bar(df_train)

In [ ]:
msno.matrix(df_train )

In [ ]:
# replace value 
df_train.branchcode = df_train.branchcode.replace({"A": 1, "B" : 2, "C" : 3, "D" : 4, "E" : 5, "F" : 6, "G" : 7, "H" : 8, "I" : 9,"K": 11, "J": 10})
df_test.branchcode = df_test.branchcode.replace({"A": 1, "B" : 2, "C" : 3, "D" : 4, "E" : 5, "F" : 6, "G" : 7, "H" : 8, "I" : 9,"K": 11, "J": 10})

### Remove naull value

In [ ]:
df_train['branchcode']=df_train['branchcode'].fillna(df_train['branchcode'].mode()[0])
df_test['branchcode']=df_test['branchcode'].fillna(df_test['branchcode'].mode()[0])

In [ ]:
msno.matrix(df_train )

In [ ]:
msno.matrix(df_test)

In [ ]:
sns.set(style = 'whitegrid')
plt.figure(figsize = (16,8))
plt.subplot(2,2,1)
sns.countplot(x = 'n_cards' , data = df_train)
plt.subplot(2,2,2)
sns.countplot(x = 'branchcode', data= df_train)
plt.subplot(2,2,3)
sns.countplot(x = 'n_cards', hue= 'good_bad_flag', data = df_train)
plt.subplot(2,2,4)
sns.countplot(x = 'loan_tenure',hue = 'good_bad_flag' , data= df_train)
plt.show()

In [ ]:
df_train.plot(figsize=(18, 8))
plt.show()

In [ ]:
plt.figure(figsize=(18, 6))
plt.title("Relation Between credit Limit vs Os Balance ")

plt.grid()
plt.scatter(df_train['creditlimit'] , df_train['os_balance'], c='k', marker='x')
plt.xlabel("Credit Limit")
plt.ylabel("Balance")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_train['good_bad_flag'], df_train['branchcode'])
plt.title("Relation Beteween branchcode and Eligiblity Status")
plt.show()

In [ ]:
corr = df_train.corr()
plt.figure(figsize=(28,12))
sns.heatmap(corr , annot = True )
plt.show()

In this heatmap, we can clearly seen the relation between two variables

### Choose ML Model
In this step, We have a lots of Machine Learning Model from sklearn package, and we need to decide which model is give us the better performance. then we use that model in final stage and send to the production level.

We will use logistic Regression 

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report,plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

First of all, we are use LogisticRegression from sklearn.linear_model package. Here is the little information about LogisticRegression.

Logistic Regression is a classification algorithm. It is used to predict a binary outcome (1 / 0, Yes / No, and True / False) given a set of independent variables. To represent binary / categorical outcome, we use dummy variables. You can also think of logistic regression as a special case of linear regression when the outcome variable is categorical, where we are using log of odds as the dependent variable.

In [ ]:
train_features=['n_cards','os_balance','os_billing','tot_cash_advance_trx','tot_retail_trx','remaining_unpaid_balance','branchcode','overlimit_pct','deliquncy_score','loan_tenure','total_trx']

In [ ]:
x_train = df_train[train_features].values
y_train = df_train['good_bad_flag'].values

x_test = df_test[train_features].values


In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train, y_train)

### Predict Model


In [ ]:
print('Coefficient of model :', logistic_model.coef_)

In [ ]:
print('Intercept of model',logistic_model.intercept_)

In [ ]:
score = logistic_model.score(x_train, y_train)
print('accuracy_score overall :', score)
print('accuracy_score percent :', round(score*100,2))

In [ ]:
# predict the target on the test dataset
predict_test = logistic_model.predict(x_test)
print('Target on test data',predict_test) 

### Deploy Model

Finally, we are done so far. The last step is to deploy our model in production map. Suppose we need to export our model and bind with web application API.
Using pickle we can export our model and store in to logistic_model.pkl file, so we can ealy access this file and calculate customize prediction using Web App API.

A little bit information about pickle:
Pickle is the standard way of serializing objects in Python. You can use the pickle operation to serialize your machine learning algorithms and save the serialized format to a file. Later you can load this file to deserialize your model and use it to make new predictions

In [ ]:
import pickle as pkl

In [ ]:
# save the model to disk
filename = 'logistic_model.pkl'
pkl.dump(logistic_model, open(filename, 'wb'))

### Conclusion 
For huge data this modle might get slow, in that case we can use XgBoos Classifer Light GBM model. Boosting algorithom will be best for this sceniro .

In [ ]:
nan